In [24]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Disable all info messages (3) or info and warning messages (2)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 


In [10]:
from tensorflow.python.client import device_lib

In [11]:
import numpy as np
import pandas as pd
import pylab as plt

from pyace import *
from tensorpotential.potentials.ace import ACE
from tensorpotential.tensorpot import TensorPotential
from tensorpotential.fit import FitTensorPotential
from tensorpotential.constants import *
from tensorpotential.utils.utilities import generate_tp_atoms

from ase import Atoms
from ase.calculators.singlepoint import SinglePointCalculator

In [95]:
cutoff = 6

potential_config = {
        # Step 0. define deltaSplineBins
        'deltaSplineBins': 0.001,

        # Step 1. specify all elements of the basis
        'elements': ['Cu'],

        # Step 2. specify embeddings for all elements, using 'ALL' or elements name keywords
        'embeddings': {'ALL': {
                'ndensity': 2,
                'fs_parameters': [1, 1, 1, 0.5],
                'npot': 'FinnisSinclairShiftedScaled',

                #'rho_core_cut': 2000000.,
                #'drho_core_cut': 250,
        },
        },
        # Step 3. specify bonds for all elements, using 'ALL', UNARY, BINARY or elements pairs
        'bonds': {'ALL': {
                'radbase': 'SBessel',
                'radparameters': [2.0],
                'rcut': cutoff,
                'dcut': 0.01,
                #'NameOfCutoffFunction': 'cos',
                #'core-repulsion': [0.0, 1.0],
        },
        },

        # Step 4. Specify BBasisFunctions list for each block using ALL, UNARY, BINARY, ..., QUINARY keywords
        # setup per-rank nradmax_by_orders and lmax_by_orders
        'functions': {
                'ALL': {
                        'nradmax_by_orders': [16, 6],
                        'lmax_by_orders': [0, 6]
                },
        }
}

In [96]:
def do_tp_atoms(row):
    at = row['ase_atoms']
    en = row['energy_corrected']
    f = row['forces']
    calc = SinglePointCalculator(at, energy=en, forces=f)
    at.set_calculator(calc)
    
    return generate_tp_atoms(at, cutoff=cutoff)

In [126]:
np.random.seed(42)

bbasisconf = create_multispecies_basis_config(potential_config,
                                             func_coefs_initializer="zero")

In [127]:
ace = ACE(bbasisconf)

tp = TensorPotential(ace, loss_specs={
        LOSS_TYPE: 'per-atom',
        LOSS_FORCE_FACTOR: 1,
        LOSS_ENERGY_FACTOR: 10,
        L1_REG: 0e-8,
        L2_REG: 0e-8
    })

In [145]:
ace.eval_atomic_energy(at.positions)

AttributeError: 'NoneType' object has no attribute 'bond_partition'

In [100]:
# https://github.com/ICAMS/python-ace/blob/master/examples/Cu-I/Cu_df1_A1_A2_A3_EV_elast_phon.pckl.gzip

df = pd.read_pickle('./Cu_df1_A1_A2_A3_EV_elast_phon.pckl.gzip', compression="gzip")

In [103]:
df['tp_atoms'] = df.apply(do_tp_atoms, axis=1)
df['tp_atoms'] 

0      {'_ind_i': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...
1      {'_ind_i': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...
2      {'_ind_i': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...
3      {'_ind_i': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...
4      {'_ind_i': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...
                             ...                        
100    {'_ind_i': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...
101    {'_ind_i': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...
102    {'_ind_i': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...
103    {'_ind_i': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...
104    {'_ind_i': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...
Name: tp_atoms, Length: 105, dtype: object

In [104]:
tpf = FitTensorPotential(tp, eager=False)

In [105]:
tpf.fit(df, niter=10, batch_size=105, optimizer='BFGS')

2022-12-13 08:45:28,087 - tensorpotential.fit - INFO - Minimizer options: {'gtol': 1e-08, 'disp': True, 'maxiter': 10}
/home/mbahng/miniconda/envs/ace/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("concat_19:0", shape=(None,), dtype=int32), values=Tensor("concat_18:0", shape=(None, 3), dtype=float64), dense_shape=Tensor("gradient_tape/Cast_7:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
/home/mbahng/miniconda/envs/ace/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/gradient_tape/Reshape_134:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/gradient_tape/Reshape_133:0", shape=(None, 16), dtype=float64), dense_shape=Tensor("gradient_tape/gradient_tape/Cast_1:0", shape=(2,),

         Current function value: 4.824717
         Iterations: 10
         Function evaluations: 18
         Gradient evaluations: 18
Fitting took  30.12 seconds


In [106]:
print(ace.get_coefs()[:10])

tf.Tensor(
[ 1.00389926e+00  4.00817046e-03  2.05781768e-03  4.25863756e-03
  4.23271645e-03 -1.09855351e-03 -1.65402930e-03  1.78998834e-03
 -1.17946734e-04 -2.85728656e-03], shape=(10,), dtype=float64)


In [ ]:
tpf.fit(df, niter=10, batch_size=105, optimizer='BFGS')

In [108]:
print(ace.get_coefs()[:10])

tf.Tensor(
[1.00224337e+00 2.97231998e-03 4.56546990e-04 1.29505709e-04
 4.44839996e-03 5.94584772e-03 2.37008695e-03 1.87937038e-03
 4.85013124e-03 2.86020449e-03], shape=(10,), dtype=float64)


In [109]:
tpf.fit(df, niter=10, batch_size=105, optimizer='BFGS')

2022-12-13 08:46:45,496 - tensorpotential.fit - INFO - Minimizer options: {'gtol': 1e-08, 'disp': True, 'maxiter': 10}
2022-12-13 08:46:46,181 - tensorpotential.fit - INFO - Iteration:  #21   (35 evals):       Loss: 0.646456 | RMSE Energy(low): 23.81 (23.81) meV/at | Forces(low): 11.77 (11.77) meV/A | Time/eval: 587.18 mcs/at
2022-12-13 08:46:46,490 - tensorpotential.fit - INFO - Iteration:  #22   (36 evals):       Loss: 0.520623 | RMSE Energy(low): 21.11 (21.11) meV/at | Forces(low): 11.95 (11.95) meV/A | Time/eval: 648.88 mcs/at
2022-12-13 08:46:46,782 - tensorpotential.fit - INFO - Iteration:  #23   (37 evals):       Loss: 0.361089 | RMSE Energy(low): 17.26 (17.26) meV/at | Forces(low): 11.46 (11.46) meV/A | Time/eval: 673.56 mcs/at
2022-12-13 08:46:47,084 - tensorpotential.fit - INFO - Iteration:  #24   (38 evals):       Loss: 0.299278 | RMSE Energy(low): 15.77 (15.77) meV/at | Forces(low): 10.18 (10.18) meV/A | Time/eval: 703.84 mcs/at
2022-12-13 08:46:47,380 - tensorpotential.fit

         Current function value: 0.166354
         Iterations: 10
         Function evaluations: 12
         Gradient evaluations: 12
Fitting took   3.39 seconds


In [112]:
print(ace.get_coefs().shape)

(998,)


In [23]:
bbasis = ace.get_updated_config()
bbasis.save('./my_fitted_potential.yaml')